In [1]:
import numpy as np
import Agent
import Environment
import Referee

In [2]:
import pandas as pd

In [3]:
df_type = pd.read_csv('type.csv', index_col=0).T
df_type

type,normal,fire,water,electric,grass,ice,fighting,poison,ground,flying,psychic,bug,rock,ghost,dragon,dark,steel,fairy
normal,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0
fire,1.0,0.5,2.0,1.0,0.5,0.5,1.0,1.0,2.0,1.0,1.0,0.5,2.0,1.0,1.0,1.0,0.5,0.5
water,1.0,0.5,0.5,2.0,2.0,0.5,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.5,1.0
electric,1.0,1.0,1.0,0.5,1.0,1.0,1.0,1.0,2.0,0.5,1.0,1.0,1.0,1.0,1.0,1.0,0.5,1.0
grass,1.0,2.0,0.5,0.5,0.5,2.0,1.0,2.0,0.5,2.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0
ice,1.0,2.0,1.0,1.0,1.0,0.5,2.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,2.0,1.0
fighting,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,2.0,0.5,0.5,1.0,1.0,0.5,1.0,2.0
poison,1.0,1.0,1.0,1.0,0.5,1.0,0.5,0.5,2.0,1.0,2.0,0.5,1.0,1.0,1.0,1.0,1.0,0.5
ground,1.0,1.0,2.0,0.0,2.0,2.0,1.0,0.5,1.0,1.0,1.0,1.0,0.5,1.0,1.0,1.0,1.0,1.0
flying,1.0,1.0,1.0,2.0,0.5,2.0,0.5,1.0,0.0,1.0,1.0,0.5,2.0,1.0,1.0,1.0,1.0,1.0


In [4]:
# 初期化

state = {
    # 処理に必要な情報
    'system_info': {
        # 現在のフェーズ
        'phase': 'phase__battle_start',
        # 現在のプロセス
        'process': '',
        # 処理順更新フラグ
        'update_battle_order_flag': False,
        # 現在のフェーズ処理対象
        'phase_target': 'A',
        # 現在のプロセス処理対象
        'process_target': 'A',
        # 処理順
        'battle_order': ['A', 'B'],
        # 行動順
        'move_order': [],
        # 計算したダメージ量
        'damage': 0
    },
    # 共通の情報
    'all_info':{
        'weather': ''
    },
    # A（自分）の情報
    'A_info': {
        # 交代待機がいない場合は -1
        'activate_pokemon_id': 0,
        # 場に誰も出ていない場合は -1
        'active_pokemon_id': -1,
        # 場に出てこれないポケモンがいない場合は、-1
        'disable_activate_pokemon_id': -1,
        # テラスタルを使用したかどうか
        'tera_orb': 'unused',
        # 行動可能かどうか
        'is_actionable': True,
        # 選択された技（選択されていない場合は-1）
        'selected_move_id': -1,
        # テラスタル化する選択をしたか
        'selected_tera': False,
        # 場に出ているポケモンが状態異常「ひんし」になった時刻
        'fainted_time': np.inf,
        'pokemons': {
            0 :{
                'name': 'ガブリアス',
                'ailment': '',
                'conditions': {},
                'is_tera': False,
                'level': 50,
                'types': ['dragon', 'ground'],
                # 'ability': "さめはだ",
                'ability': "いかく",
                'tera_type': 'ground',
                # テラスタル化していて、かつ元タイプと一致しているかどうか
                'tera_type_match': False,
                'item': 'こだわりハチマキ',
                'max_H': 183,
                'H': 183,
                'A': int(200 * 1.5),
                'B': 116,
                'C': 90,
                'D': 105,
                'S': 154,
                'A_rank': 0,
                'critical_rank': 0,
                'accuracy_rank': 0,
                'moves': {
                    0: {
                        'name': 'げきりん',
                        'priority': 0,
                        'is_attack': True,
                        'move_category': 'physical',
                        # 'type': 'dragon',
                        'type': 'bug',
                        'critical_rank': 0,
                        'accuracy': 100,
                        'power': 120,
                        'PP': 10,
                        # 連続攻撃かどうか（2~5回ランダム）
                        'is_combo': False,
                        # 接触攻撃かどうか
                        'is_contact': True
                    },
                    1: {
                        'name': 'じしん',
                        'priority': 0,
                    },
                    2: {
                        'name': 'アイアンヘッド',
                        'priority': 0,
                    },
                    3: {
                        'name': 'ねごと',
                        'priority': 0,
                    }
                }
            },
            1: {
                'name': 'サーフゴー',
                'ailment': ''
            },
            2: {
                'name': 'ドドゲザン',
                'ailment': ''
            }
        }
    },
    # B（相手）の情報
    'B_info': {
        # 交代待機がいない場合は -1
        'activate_pokemon_id': 0,
        # 場に誰も出ていない場合は -1
        'active_pokemon_id': -1,
        # 場に出てこれないポケモンがいない場合は、-1
        'disable_activate_pokemon_id': -1,
        # テラスタルを使用したかどうか
        'tera_orb': 'unused',
        # 行動可能かどうか
        'is_actionable': True,
        # 選択された技（選択されていない場合は-1）
        'selected_move_id': -1,
        # テラスタル化する選択をしたか
        'selected_tera': False,
        # 場に出ているポケモンが状態異常「ひんし」になった時刻
        'fainted_time': np.inf,
        'pokemons': {
            0 :{
                'name': 'ミミッキュ',
                'ailment': '',
                'conditions': {},
                'is_tera': False,
                'level': 50,
                'types': ['ghost', 'fairy'],
                'ability': 'ばけのかわ',
                'tera_type': 'fairy',
                # テラスタル化していて、かつ元タイプと一致しているかどうか
                'tera_type_match': False,
                # 'item': 'いのちのたま',
                'item': '',
                'max_H': 1310,
                'H': 1310,
                'A': 156,
                'B': 100,
                'C': 63,
                'D': 125,
                'S': 148,
                'A_rank': 0,
                'critical_rank': 0,
                'accuracy_rank': 0,
                'moves': {
                    0: {
                        'name': 'じゃれつく',
                        'priority': 0,
                    },
                    1: {
                        'name': 'シャドークロー',
                        'priority': 0,
                    },
                    2: {
                        'name': 'かげうち',
                        'priority': 1,
                        'is_attack': True,
                        'move_category': 'physical',
                        'type': 'ghost',
                        'critical_rank': 0,
                        'accuracy': 100,
                        'power': 40,
                        'PP': 30
                    },
                    3: {
                        'name': 'つるぎのまい',
                        'priority': 0,
                    }
                }
            },
            1: {
                'name': 'ギャラドス',
                'ailment': ''
            },
            2: {
                'name': 'セグレイブ',
                'ailment': ''
            }
        }
    }
}

action = -1

In [5]:
# 対戦ループ

for _ in range(100):
    # エージェントの行動と現在の状態から、次の状態を決定
    state, request = Environment.update_state(state, action)


    # エージェントは現在の状態から、行動を決定
    if request == 'A':
        action = Agent.response_action(state, request)
    elif request == 'B':
        action = Agent.response_action(state, request, auto=True)

    # 終了判定
    elif request == 'end_judgement':
        if Referee.judge_end(state):
            break
    else:
        Exception
    
    # デバッグ用
    if action == 'debug':
        break

comment: 対戦開始！
comment: プレイヤーA は ガブリアス を繰り出した！
comment: プレイヤーB は ミミッキュ を繰り出した！
comment: プレイヤーB の ミミッキュ は いかく されて 攻撃が １だんかい　さがった。

----------現在の状態----------
-----全体-----
天候: 

-----プレイヤーA-----
ポケモン	: ガブリアス (Lv:50)
タイプ	: ['dragon', 'ground']
ﾃﾗｽﾀﾙ化	: False
HP	: 183/183
状態異常	: 

控え匹数	: 2

-----プレイヤーB-----
ポケモン	: ミミッキュ (Lv:50)
タイプ	: ['ghost', 'fairy']
ﾃﾗｽﾀﾙ化	: False
HP	: 1310/1310
状態異常	: 

控え匹数	: 2


----------
プレイヤーA の行動を選択してください(半角英数字記号)：
テラスタル化したい場合は、選択肢の後ろに「_tera」と入力してください

{'move_0': 'げきりん', 'move_1': 'じしん', 'move_2': 'アイアンヘッド', 'move_3': 'ねごと'}
{'pokemon_1': 'サーフゴー', 'pokemon_2': 'ドドゲザン'}



actionA> debug


デバッグモードにより強制終了します。


In [ ]:
state['A_info']